## Problems
    1. Fix the download pdf from link function
        1.1 file stuck at download
        1.2 broken file download
    2. Turn it int OOP 

## importing libraries

In [1]:
# from googlesearch import search
import webbrowser
import re
import urllib.request
import requests
import bs4
from fake_useragent import UserAgent

In [2]:
import pandas as pd
import numpy as np
from fake_useragent import UserAgent
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup

In [3]:
import re
import os

In [4]:
from googlesearch import search

## Reading the list

In [8]:
def get_comp_list(path = "Top 500 companies.txt"):
    with open(path, "r") as f:
        comp_list = f.readlines()
    return comp_list

## Searching PDF with no block(new)

In [9]:
def google_results(keyword, n_results=10):
    query = keyword
    query = urllib.parse.quote_plus(query) # Format into URL encoding
    number_result = n_results
    ua = UserAgent()
    google_url = f"https://www.google.com/search?q= {query} &num= + {number_result}"
    response = requests.get(google_url, {"User-Agent": ua.random})
    soup = BeautifulSoup(response.text, "html.parser")
    links = get_links(soup)
    titles = get_title_list(soup)
    desc_list = get_desc(soup)
    lst = []
    for (l,t,d) in zip(links,titles,desc_list):
        lst.append({"title" : t, "link" : l, "desc" : d})
    return lst[:number_result]

def get_links(soup):
    result = soup.find_all('div', attrs = {'class': 'ZINbbc'})
    pattern = '\/url\?q\=(.*)\&sa'
    results = []
    for i in result:
        if "url" in str(i):
            text = str(i.find('a', href = True)['href'])
            results.append(re.search(pattern,text))
    #group(1) returns the text of the match object
    links=[i.group(1) for i in results if i != None]
    return links

def get_title_list(soup):
    result = soup.find_all('h3', attrs = {'class': 'zBAuLc'})
    titles = [i.text for i in result]
    return titles

def get_desc(soup):
    result = soup.findAll(True, {'class':"kCrYT"})
    desc_list = []
    for i,r in enumerate(result):
        if i % 2 != 0:
            desc_list.append(r.text)     
    return desc_list



In [10]:
def search_and_get_pdf(n = None,year = None, c=None):
    if c == None:
        c = int(input("Enter\n1. Normal Downlaod\n2. Redownload \n"))
    if c == 1:
        comp_list = get_comp_list()
        if year == None:
            year = int(input("Enter year : "))
        if n == None:
            n = int(input("How many companies would you like to search? : "))
            comp_subset = comp_list[:n]
    elif c == 2:
        comp_subset = get_redownload_list(year)
    else:
        return None
    
    print(f"Initiating search of Annual reports for the {year}......\n")
    dic = {}
    for x in comp_subset:
        comp = x.strip()
        #if the pdf of a year exist
        if os.path.exists(f"pdf/{year}") == False:
            os.mkdir(f"pdf/{year}")
        if c == 1:
            if os.path.exists(f"pdf/{year}/{comp}"):
                if os.listdir(f"pdf/{year}/{comp}") != []:
                    print(f"{comp} already downloaded! Skipping search!")
                    continue
        
#       query = f"{comp} annual report site:\"https://www.bseindia.com\" filetype:pdf intext:\"{year} {year+1}\" " 
#         query = f"{comp} bse annual report {year}-{year+1} filetype:pdf intext:\"{comp}\" "
        query = f"{comp} bse annual report {year} filetype:pdf intext:\"{comp}\" "

#             query2 = f"{comp} annual report 2019-20 intext:\"Do you have a policy\" filetype:pdf"
        print(f"\nLinks for {comp} --------------------------------------------\n")
        links = []
        link = ""
        title = ""
        desc = ""
        result = google_results(query,1)
        for r in result:
            title = r["title"]
            desc = r["desc"]
            link = r["link"]
            print(f"\nTitle : {color.BOLD}{title}{color.END}")
            print(f"Description : {desc}")
            print(f"link :{link}")
#             if f"{year-1}" in desc or f"{year-1}" in title:
#                 print("SKIPPING.......")
#                 continue
            links.append(link)
#             break
        dic[x] = links
    return dic

In [11]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

print(color.BOLD + 'Hello World !' + color.END)

Hello World !


## Downloading PDF from the list of links

In [12]:
def get_filesize(download_path):
    file_size = os.path.getsize(f"{download_path}")
    return file_size

In [13]:
def validate_links(file_name):
    pass

In [14]:
def get_filename(y):
    #check for pdf extension in link if not then add
    if re.findall("pdf", y) == []:
        file_name = os.path.split(y)[1] + ".pdf"
    else:
        file_name = os.path.split(y)[1]
    ##check for pdf in filename
    if re.findall("pdf",file_name) == []:
        y = y.replace(file_name,"")
        file_name = os.path.split(y[:-1])[1]
    
    i = file_name.find("pdf")
    if i > 0:
        file_name =  file_name[:i+3]
        
    ##check for special characters
    pattern= "'[@!#$^&*%<>?/\\|}{~:]'\"="
    
    for x in pattern:
        n = file_name.replace(x,"")
        file_name = n
    return n

In [15]:
##download directly
def download_pdf_direct(y,file_name,year,company_name):
    response = requests.get(y)
    pdf = open(f"pdf/{year}/{company_name}/{file_name}", 'wb')
    pdf.write(response.content)
    pdf.close()
    print(f"\t{file_name} downloaded")

In [16]:
##download at each folder
def download_all_pdf(dic,year = None):
    if year == None:
        year = int(input("Enter year : "))
    error_files = []
    for x in dic:
        company_name = x.strip()
        link_of_pdfs = dic[x]
        path = "pdf"
        download_path = f"{path}/{year}/{company_name}"
        #go to pdf and create folder
        if os.path.exists(f"{path}/{year}") == False:
            os.mkdir(f"{path}/{year}")
        if os.path.exists(download_path) == False:
            os.mkdir(download_path)
        f_size = get_filesize(f"{download_path}")
        if os.listdir(download_path) == [] or f_size < 10000:
            print(f"Downloading report for {company_name}")
            for y in link_of_pdfs:
                #gets the pdf name from the link
                file_name = get_filename(y)
                try:
#                     print(f"{y}\n{file_name}\n{company_name}")
                    download_pdf_direct(y,file_name,year,company_name)
                except:
                    print(f"!!!!!------------Some error occured while downloading {file_name}")
                    error_files.append(company_name)
            print(f"All {company_name} report downloaded")
    
    print("----------------------------------------------------------------------------------")
    print("Download complete!")
    
    
    if error_files != []:
        print("List of company not downloaded")
        with open(f"pdf/comp_{year}.txt","w") as f:
            for x in error_files:
                print(f"\t{x}")
                f.write(x+"\n")

## Start search and download

In [32]:
def start_it():
    y = int(input("Enter year :"))
    choice = ""
    if os.path.exists(f"pdf/{y}"):
        check_files(y)
        choice = input("Do you want to download? y or n :")
        
    if choice == "y":
        comp_dict = search_and_get_pdf(None,y,2)
    else:
        comp_dict = search_and_get_pdf(None,y,1)
    
    download_all_pdf(comp_dict,y)

## Validate downloaded files

## Check files

In [27]:
def check_files(year):
    with open(f"pdf/broken_files_{year}.txt","w") as f:
        for i in os.listdir(f"pdf/{year}/"):
            path = f"pdf/{year}/{i}/"
            file = os.listdir(path)
            if file == []:
                continue
            file_path = path+file[0]
            file_size = os.path.getsize(file_path)
    #         print(f"{i} File Size is :{file_size} bytes")
            if file_size < 10000:
                f.write(f"{i}\n")
                print(f"{i} file size is too small {file_size} bytes\n")


In [19]:
check_files(2018)

Bank of India file size is too small 345 bytes

Canara Bank file size is too small 345 bytes

Coal India file size is too small 345 bytes

GAIL (India) file size is too small 6 bytes

General Insurance Corporation of India file size is too small 345 bytes

Hindustan Unilever file size is too small 0 bytes

Jindal Steel & Power file size is too small 345 bytes

Larsen & Toubro file size is too small 0 bytes

Mahindra & Mahindra file size is too small 345 bytes

NTPC file size is too small 345 bytes

Oil & Natural Gas Corporation file size is too small 43 bytes

Power Grid Corporation of India file size is too small 0 bytes

Reliance Industries file size is too small 345 bytes

State Bank of India file size is too small 345 bytes

Steel Authority of India file size is too small 345 bytes

Tech Mahindra file size is too small 0 bytes

UltraTech Cement file size is too small 345 bytes

Union Bank of India file size is too small 345 bytes

Vedanta file size is too small 345 bytes



## Get redownload list

In [20]:
def get_redownload_list(year):
    path = f"pdf/broken_files_{year}.txt"
    with open(path, "r") as f:
        r_list = f.readlines()
    return r_list

In [35]:
start_it()

Enter year : 2020
How many companies would you like to search? :  5


Initiating search of Annual reports for the 2020......


Links for Reliance Industries --------------------------------------------


Title : [PDF] Annual Report - BSE
Description : 31-Mar-2020 · Reliance Industries Limited. Integrated Annual Report 2019-20. Naye India Ka Naya Josh. RELIANCE RETAIL. Reliance Retail continues to grow ...
link :https://www.bseindia.com/bseplus/AnnualReport/500325/5003250320.pdf

Links for Indian Oil Corporation --------------------------------------------


Title : [PDF] Integrated Annual Report 2019-20 - Iocl.com
Description : 21-Aug-2020 · Indian Oil Corporation Limited. 3rd Integrated Annual Report. 2. 3. ABOUT THE REPORT. Reporting Approach. With extensive petroleum refining, ...
link :https://iocl.com/download/IndianOil-Annual-Report-2019-20.pdf

Links for Oil & Natural Gas Corporation --------------------------------------------


Title : [PDF] Annual Report 2018-19 - ONGC
Description : 25-Jul-2019 · dedicated energy soldiers of Oil & Natural Gas C

In [36]:
check_files(2020)

Oil & Natural Gas Corporation file size is too small 43 bytes

Reliance Industries file size is too small 345 bytes

State Bank of India file size is too small 245 bytes



## Test download